## G-CARE Benchmark

In [1]:
using Graphs: DiGraph
using StatsPlots
include("Source/datasets.jl")
include("Source/QuasiStableCardinalityEstimator.jl")

get_exact_size (generic function with 1 method)

In [2]:
aids_data_file_path = "dataset/aids/aids.txt"
human_data_file_path = "dataset/human/human.txt"
lubm80_data_file_path = "dataset/lubm80/lubm80.txt"
yago_data_file_path = "dataset/yago/yago.txt"

aids_data = load_dataset(aids_data_file_path)
#human_data = load_dataset(human_data_file_path)
#lubm80_data = load_dataset(lubm80_data_file_path)
#yago_data = load_dataset(yago_data_file_path)
nothing

In [3]:
#datasets = [aids_data, human_data, lubm80_data, yago_data]
#dataset_names = ["aids", "human", "lubm80", "yago"]
datasets = [aids_data]
dataset_names = ["aids"]

1-element Vector{String}:
 "aids"

In [4]:
build_time = []
summary_size = []
color_summaries = []
for dataset in datasets
    results = @timed generate_color_summary(dataset, 64, verbose=true)
    push!(build_time, results[2])
    push!(summary_size,  get_color_summary_size(results[1]))
    push!(color_summaries, results[1])
end

making cycles
making paths


making cycles
making paths


making cycles


making paths
Started coloring


Finished coloring
Started bloom filters
Finished bloom filters
Started cardinality counts


Finished cardinality counts
Started tracking statistics

Finished tracking statistics


In [5]:
lubm80_query_paths = ["lubm80/lubm80_Q2.txt",
"lubm80/lubm80_Q4.txt",
"lubm80/lubm80_Q7.txt",
"lubm80/lubm80_Q8.txt",
"lubm80/lubm80_Q9.txt",
"lubm80/lubm80_Q12.txt"
                    ]
exactSizes = []
bounds = []
relativeErrors = []
println("Summary Size: ", summary_size[1])
println("Summary Build Time: ", build_time[1])
for query_path in lubm80_query_paths
    println("Query: ", query_path)
    id_and_query = load_query("queryset/" * query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[1])
    exact_size_results = @timed get_exact_size(query, datasets[1])
    gcare_size = load_true_cardinality("TrueCardinalities/" * query_path)
    println("Bound Time: ", bound_results[2])
    println("Exact Size Time: ", exact_size_results[2])
    println("Exact Size: ", exact_size_results[1])
    println("GCare Exact Size: ", gcare_size)
    if gcare_size != exact_size_results[1]
        println("Bad Exact Size Estimate!!")
        break
    end
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", (bound_results[1])./exact_size_results[1])
    push!(exactSizes, gcare_size)
    push!(bounds, bound_results[1])
    push!(relativeErrors, bound_results[1] ./ gcare_size)
end

Summary Size: 50886


Summary Build Time: 344.7648349


Query: lubm80/lubm80_Q2.txt


Bound Time: 0.2970543
Exact Size Time: 0.0435462
Exact Size: 0
GCare Exact Size: 212
Bad Exact Size Estimate!!


In [ ]:
human_query_directories = ["/queryset/human/Chain_3/",
"/queryset/human/Graph_3/",
"/queryset/human/Star_3/",
"/queryset/human/Tree_3/",
                    ]
human_query_paths = [readdir(pwd() * dir, join=true) for dir in human_query_directories]
human_query_paths = [(human_query_paths...)...]
human_exact_sizes = []
human_bounds = []
human_relative_errors = []
println("Summary Size: ", summary_size[1])
println("Summary Build Time: ", build_time[1])
for query_path in human_query_paths
    println("Query: ", query_path)
    id_and_query = load_query(query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[1])
    exact_size_results = @timed get_exact_size(query, datasets[1])
    gcare_size = load_true_cardinality(replace(query_path, "queryset"=>"TrueCardinalities"))
    println("Bound Time: ", bound_results[2])
    println("Exact Size Time: ", exact_size_results[2])
    println("Exact Size: ", exact_size_results[1])
    println("GCare Exact Size: ", gcare_size)
    if gcare_size != exact_size_results[1]
        println("Bad Exact Size Estimate!!")
        break
    end
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", bound_results[1]./exact_size_results[1])
    bound_results[1][2] = max(1, bound_results[1][2])
    push!(human_exact_sizes, gcare_size)
    push!(human_bounds, bound_results[1])
    push!(human_relative_errors, bound_results[1] ./ gcare_size)
end

In [ ]:
using Statistics
println(maximum([log10(x[2]) for x in human_relative_errors]))
println(mean([log10(x[2]) for x in human_relative_errors]))
println(minimum([log10(x[2]) for x in human_relative_errors]))

In [22]:
using Statistics
aids_query_directories = ["/queryset/aids/Chain_3/",
"/queryset/aids/Chain_6/",
"/queryset/aids/Chain_9/",
"/queryset/aids/Chain_12/",
"/queryset/aids/Cycle_3/",
"/queryset/aids/Cycle_6/",
"/queryset/aids/Flower_6/",
"/queryset/aids/Flower_9/",
"/queryset/aids/Flower_12/",
"/queryset/aids/Graph_3/",
"/queryset/aids/Graph_6/",
"/queryset/aids/Graph_9/",
"/queryset/aids/Graph_12/",
"/queryset/aids/Petal_6/",
"/queryset/aids/Petal_9/",
"/queryset/aids/Petal_12/",
"/queryset/aids/Star_3/",
"/queryset/aids/Star_6/",
"/queryset/aids/Star_9/",
"/queryset/aids/Tree_3/",
"/queryset/aids/Tree_6/",
"/queryset/aids/Tree_9/",
"/queryset/aids/Tree_12/"]
aids_query_paths = [readdir(pwd() * dir, join=true) for dir in aids_query_directories]
aids_query_paths = [(aids_query_paths...)...]
aids_exact_sizes = []
aids_bounds = []
aids_relative_errors = []
println("Summary Size: ", summary_size[1])
println("Summary Build Time: ", build_time[1])
for query_path in aids_query_paths
    println("Query: ", query_path)
    id_and_query = load_query(query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[1])
    gcare_size = load_true_cardinality(replace(query_path, "queryset"=>"TrueCardinalities"))
    # println("Bound Time: ", bound_results[2])
    # println("Exact Size: ", gcare_size)
    # println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    # println("Relative Error [Lower, Avg, Upper]: ", bound_results[1]./gcare_size)
    bound_results[1][2] = max(1, bound_results[1][2])
    push!(aids_exact_sizes, gcare_size)
    push!(aids_bounds, bound_results[1])
    push!(aids_relative_errors, bound_results[1] ./ gcare_size)
end
println(maximum([log10(x[2]) for x in aids_relative_errors]))
println(mean([log10(x[2]) for x in aids_relative_errors]))
println(minimum([log10(x[2]) for x in aids_relative_errors]))

In [11]:
using Statistics
println(maximum([log10(x[2]) for x in aids_relative_errors]))
println(mean([log10(x[2]) for x in aids_relative_errors]))
println(minimum([log10(x[2]) for x in aids_relative_errors]))

2.6686015167651913
-1.3494790192246389


-5.504089068119823


In [6]:
using Statistics
aids_query_directories = ["/queryset/aids/Chain_3/",
"/queryset/aids/Chain_6/",
"/queryset/aids/Chain_9/",
"/queryset/aids/Chain_12/",
"/queryset/aids/Cycle_3/",
"/queryset/aids/Cycle_6/",
"/queryset/aids/Flower_6/",
"/queryset/aids/Flower_9/",
"/queryset/aids/Flower_12/",
"/queryset/aids/Graph_3/",
"/queryset/aids/Graph_6/",
"/queryset/aids/Graph_9/",
"/queryset/aids/Graph_12/",
"/queryset/aids/Petal_6/",
"/queryset/aids/Petal_9/",
"/queryset/aids/Petal_12/",
"/queryset/aids/Star_3/",
"/queryset/aids/Star_6/",
"/queryset/aids/Star_9/",
"/queryset/aids/Tree_3/",
"/queryset/aids/Tree_6/",
"/queryset/aids/Tree_9/",
"/queryset/aids/Tree_12/"]
aids_query_paths = [readdir(pwd() * dir, join=true) for dir in aids_query_directories]
aids_query_paths = [(aids_query_paths...)...]
aids_exact_sizes = []
aids_bounds = []
aids_relative_errors = []
println("Summary Size: ", summary_size[1])
println("Summary Build Time: ", build_time[1])
for query_path in aids_query_paths
    println("Query: ", query_path)
    id_and_query = load_query(query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[1], usingStoredStats = true)
    println(bound_results[1])
    gcare_size = load_true_cardinality(replace(query_path, "queryset"=>"TrueCardinalities"))
    # println("Bound Time: ", bound_results[2])
    # println("Exact Size: ", gcare_size)
    # println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    # println("Relative Error [Lower, Avg, Upper]: ", bound_results[1]./gcare_size)
    bound_results[1][2] = max(1, bound_results[1][2])
    push!(aids_exact_sizes, gcare_size)
    push!(aids_bounds, bound_results[1])
    push!(aids_relative_errors, bound_results[1] ./ gcare_size)
end
println(maximum([log10(x[2]) for x in aids_relative_errors]))
println(mean([log10(x[2]) for x in aids_relative_errors]))
println(minimum([log10(x[2]) for x in aids_relative_errors]))

Summary Size: 50886
Summary Build Time: 344.7648349
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_0_2.txt


[0.0, 3.0253029331868264, 88272.0]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_1_1.txt
[0.0, 6.626766347820973, 178119.0]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_1_2.txt


[0.0, 10.71722139616455, 273927.0]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_1_3.txt
[0.0, 23.24952349097013, 1.371895e6]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_1_4.txt
[0.0, 76.72032235995773, 2.111905e6

]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_1_5.txt
[0.0, 6.626766347820973, 178119.0]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_1.txt
[0.0, 23.470014788551005, 16087.0]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_10.txt
[0.0, 23.470014788551005, 16087.0]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_11.txt
[0.0, 2107.055141656889, 5.2718799e7]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_12.txt
[0.0, 2107.055141656889, 5.2718799e7]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_13.txt
[0.0, 81.62644800658744, 30240.0]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_14.txt
[0.0, 81.62644800658744, 30240.0]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Co

[0.0, 2107.055141656889, 5.2718799e7]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_3.txt
[0.0, 2107.055141656889, 5.2718799e7]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_4.txt
[0.0, 2107.055141656889, 5.2718799e7]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_5.txt
[0.0, 2107.055141656889, 5.2718799e7]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_6.txt
[0.0, 694.5089551488912, 521847.0]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_7.txt
[0.0, 2107.055141656889, 5.2718799e7]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_8.txt
[0.0, 33097.37548325221, 1.245302721e9]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_2_9.txt
[0.0, 384.3902408356008, 2.5892053e7]
Query: c:\U

...

[0.0, 79.85742043988213, 9.041134078701176e18]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Tree_12/uf_Q_5_5.txt
[0.0, 23717.775685195516, 6.204311886174761e19]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Tree_12/uf_Q_5_6.txt


[0.0, 1338.477907798164, 8.410607670692251e19]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Tree_12/uf_Q_5_7.txt


[0.0, 732782.4741047742, 1.5993016320693008e22]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Tree_12/uf_Q_5_8.txt
[0.0, 60.614257752924594, 1.8434287288925422e19]
Query: c:\Users\diand\OneDrive\Desktop\Cardinality-with-Colors/queryset/aids/Tree_12/uf_Q_5_9.txt
[0.0, 482.84014813720347, 4.739280399759701e19]


4.039348293277238
-0.8945893074621114


-5.456417659474337


In [8]:
println(maximum([log10(x[2]) for x in aids_relative_errors]))
println(mean([abs(log10(x[2])) for x in aids_relative_errors]))
println(minimum([log10(x[2]) for x in aids_relative_errors]))

4.039348293277238


1.4275131145459987


-5.456417659474337


In [ ]:
lubm80_query_paths = ["lubm80/lubm80_Q2.txt",
# "lubm80/lubm80_Q4.txt",
# "lubm80/lubm80_Q7.txt",
# "lubm80/lubm80_Q8.txt",
# "lubm80/lubm80_Q9.txt",
# "lubm80/lubm80_Q12.txt"
                    ]
exactSizes = []
bounds = []
relativeErrors = []
println("Summary Size: ", summary_size[1])
println("Summary Build Time: ", build_time[1])
for query_path in lubm80_query_paths
    println("Query: ", query_path)
    id_and_query = load_query("queryset/" * query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[1])
    exact_size_results = @timed get_exact_size(query, datasets[1])
    gcare_size = load_true_cardinality("TrueCardinalities/" * query_path)
    println("Bound Time: ", bound_results[2])
    println("Exact Size Time: ", exact_size_results[2])
    println("Exact Size: ", exact_size_results[1])
    println("GCare Exact Size: ", gcare_size)
    if gcare_size != exact_size_results[1]
        println("Bad Exact Size Estimate!!")
        break
    end
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", (bound_results[1])./exact_size_results[1])
    push!(exactSizes, gcare_size)
    push!(bounds, bound_results[1])
    push!(relativeErrors, bound_results[1] ./ gcare_size)
end

In [12]:
relativeErrors

Any[]

In [ ]:
exactSizes